# 内核基础
* 进程状态：
    <table>
        <tr>
            <td><img src='../images/devops/进程状态.png'></td>
            <td><img src='../images/devops/进程状态转换.png'></td>
        </tr>
    </table>
* 进程创建：
    * fork()：通过系统clone调用实现，通过一系列参数标志指明父子进程需要共享的资源，然后通过clone方法里面的do_fork创建真实的线程
    * vfork()：与fork的区别是不拷贝父进程的页表项，父进程将会被子进程阻塞
    * ```clone(SIGCHLD, 0)```创建fork进程，```clone(CLONE_VFORK | CLONE_VM | SIGCHLD, 0)```创建vfork进程，因此在clone是传递不同的参数可以创建不同的进程，以及线程（linux中没有独立的线程概念，每个线程都是一个特殊的进程），各个标志的详细信息参考《Linux内核设计与实现》第46页
* 孤儿进程：指父进程比子进程先结束，并没有为子进程找到新的父进程，此时结束子进程，子进程被执行系统调用do_exit，处于僵死状态，但其进程状态还在内存中，并且无法被清除，因为清除步骤跟结束步骤是分开的，只有子进程结束后，然后父进程告诉系统不需要该子进程的信息了，子进程的状态才会被销毁
    * 在当前线程组中找一个作为该进程的父进程，或者找init做它们的父进程
* 进程调度：
    * 多任务系统：抢占式多任务和非抢占式多任务
    * IO消耗型进程和处理器消耗型进程，响应速度和吞吐量之间做平衡，响应速度快更适合IO消耗型进程，如Linux
    * 默认优先级调度：ps -el中的NI表示nice值，优先级越高nice值越低，nice值范围是-20~19；实时优先级调度：优先级范围0~99，值越大优先级越高，ps -eo * 查看；优先级值要映射到时间片上，也就是默认分配给进程的单位时间
    * O(1)调度程序：
        * 反转楼梯最后期限调度算法(Rotating Staircase Deadline Scheduler)，更名为完全公平调度算法(CFS)：时间记账，记录进程执行时间，称为vruntme虚拟实时；然后通过红黑树选择最小vruntime值的进程作为下一执行进程，红黑树最左边叶子节点即为最小值；进程睡眠会从红黑树中移除，添加到休眠队列中；进程唤醒则是从休眠队列中移除，添加到红黑树中
* 系统调用处理结构图：<img src='../images/devops/系统调用结构.png' width='350px'>
* Linux中的数据结构：
    * 双向循环列表
    * 生产者-消费者队列
    * 映射，映射ID-指针
    * 红黑树，自平衡二叉树
    * 算法时间复杂度：$O(1), O(\lg{n}), O(n), O(n^2), O(n^3), O(2^n), O(n!)$
* 将列表节点塞进数据结构中：
    <table>
        <tr>
            <td><img src='../images/devops/列表节点1.png'></td>
            <td><img src='../images/devops/列表节点2.png'></td>
            <td><img src='../images/devops/列表节点3.png'></td>
        </tr>
    </table>
* 中断：中断处理程序，设备的中断处理程序是设备驱动程序的一部分，设备驱动程序就是管理设备的内核代码
    * 中断处理程序分为上半部和下半部（软中断，tasklets，任务队列）
    * 上半部处理快速响应任务，如网卡收到网络数据产生中断，马上内核就会执行网卡中断程序的上半部拷贝网卡中的数据到内核中
    * 下半部处理中断处理密切相关但是中断处理程序本身不执行的工作
* 内核同步机制：原子操作，自旋锁（排他锁），读-写自旋锁，信号量，读-写信号量，互斥体，完成变量，大内核锁，顺序锁
* 内存管理：
    * 虚拟内存最小内存管理单位(MMU)为页，页结构就是一个简单的结构体，包含相关状态变量以及实际的物理内存地址指针，该页结构只是做物理内存的映射，并不真正的存储数据，操作系统内核通过该页结构管理整个物理内存，操作系统提供给应用程序的内存就是这些页结构，页大小为4KB和8KB
    * 由于物理内存有些地方存在特殊限制，因此页又被分为不同的区，形成内存池，然后根据应用程序的需求，分配不同功能的内存给应用程序使用
    * 用户空间和内核空间：
        * 用户空间指常规进程所在的区域，该区域的代码不能直接访问硬件设备，同理硬件设备不能直接访问用户空间的数据
        * 内核空间指操作系统所在的区域，能与设备空间器通讯，控制用户区域的进程运行状态，并且所有的I/O操作直接或者间接通过内核空间完成
    * 虚拟内存：操作系统提供给内核空间和用户空间使用
        * 一个以上的虚拟地址可以指向同一个物理内存地址
        * 虚拟内存空间可大于实际可用的硬件内存
        * 内存页面调度过程：
            * CPU引用某内存地址时，MMU负责确定该地址所在页，并将虚拟页号转换为物理页号
            * 当MMU找不到虚拟页号对应的物理页号时，向CPU报错
            * CPU接收到错误时，验证该虚拟页的有效性
                * 如果该虚拟页有效，则会将对应的物理页从磁盘调入内存，并按照一定规则将其他物理页刷入磁盘
                * 如果虚拟页无效，则会产生段错误，应用程序直接强制退出
            * MMU根据调度结果，刷新虚拟页到物理页的映射
        * 虚拟内存页调度下的I/O操作：
            * 确定请求数据分布在系统的哪些页
            * 在内核分配足够的虚拟内存页
            * 将虚拟内存页与磁盘文件系统页建立映射
            * 为虚拟内存页发起页调度请求，将虚拟内存对应的物理页更新到内存中，并更新MMU中的虚拟页到物理页的映射
            * 页调度完成后，将数据加载到虚拟页对应的物理页中，将数据读入内存
    * vmalloc，kmalloc，kfree，alloc_pages
* 进程地址空间：
    * 进程地址空间是由操作系统内核提供的虚拟地址空间，是一段连续的空间，但是内核会为这段连续的空间分段（共享这段区域的进程就叫线程）：
        * 可执行文件的内存映射，代码段
        * 已初始化全局变量的内存映射，数据段
        * 未初始化全局变量的内存映射，bss段
        * 进程空间栈的内存映射
        * 动态链接库的内存映射
        * 任何内存映射文件
        * 任何共享内存段
        * 任何匿名的内存映射
* 虚拟文件系统VFS：文件，目录，索引点(inode)，挂载点
    * 文件系统需要被安装到特定的挂载点
    * 索引点存储了文件相关的元数据信息，如控制权限，大小，拥有者，创建时间等
    * 四个对象：
        * 超级块对象代表已安装的文件系统（内核针对不同文件系统的操作方法write_inode, sync_fs）：对应于磁盘特定扇区中的文件系统超级块或文件系统控制块
        * 索引节点对象代表具体的文件（create, link）：包含内核操作文件或目录的全部信息
        * 目录项对象代表目录（d_compare, d_delete）：包含有效的索引节点，并且复杂路劲解析，有被使用的，最近使用的缓存链表
        * 文件对象代表进程打开的文件（read, write）：应用操作的真实对象
* IO设备操作最小单位是块，一般是512字节、1KB、4KB，对应于物理磁盘的最小操作单位扇区，不能小于设备扇区大小，不能大于内存页大小
    * 内存中的缓冲区是磁盘块在物理内存中的表示，也就是将磁盘映射到物理内存中
* 环回文件系统（loopback, AWS snapshot实现原理）：通常是在设备上（例如磁盘分区）创建文件系统，这些存储设备能够以设备文件的形式来使用，比如 /dev/device_name。为了使用存储设备上的文件系统，我们需要将其挂载到一些被称为挂载点（mount point）的目录上。环回文件系统是指那些在文件中而非物理设备中创建的文件系统，将这些文件作为文件系统挂载到挂载点上。

****

# 文件操作
* cat：```-s, -T, -n```
* tac：逆序打印文件
* cut：按列切分文件，```cut -f2 -d";" delimited_data.txt```
* paste：按列合并文件，```paste file1.txt file2.txt -d ","```
* mktemp：穿件临时文件，-d表示创建临时目录
* rename：文件重命名
* split：分割文件，```split -b 10k data.file -d -a 4 split_file``` 
* ```look word filepath; grep "^word" filepath```：找出filepath中以word开头的单词
* <b>dd：会克隆给定的输入内容，然后将一模一样的一份副本写入到输出，```dd if=/dev/zero of=junk.data bs=1M count=1```，bs代表以字节为单位的块大小，count代表需要被复制的块数</b>
* <b>comm：对比两个文件，```comm A.txt B.txt```，输出结果中，第一列只出现在A.txt中，第二列只出现在B.txt中，第三列是两个文件相同的地方，-n则表示不输出第n列，```comm A.txt B.txt -1 -2```表示只输出第三列</b>
* <b>chmod和chown：文件权限以及所有人管理</b>
    * ```chmod u=rwx g=rw o=r filename -R; chmod o+x filename; chmod a-x filename; chmod 764 filename```
    * ```chown user.group filename -R; chown slynux.slynux test.sh```
* chattr：将文件设置为不可修改，```chattr +i file; chattr -i file```
* touch：创建空白文件
* <b>ln：创建文件链接，```ln -s target symbolic_link_name```</b>
    * 硬链接：A hard link then just creates another file with a link to the same underlying inode，复制一份
    * 软链接：A symbolic link is a link to another name in the file system，快捷方式
* <b>diff和patch(git做版本比较的基础)：```diff -Naur directory1\file1 directory2\file2```找出文件或目录中的不同，```diff -u version1.txt version2.txt > version.patch;  patch -p1 version1.txt < version.patch```将version1.txt文件恢复成version2.txt</b>
* <b>head和tail：打印文件头部或尾部几行代码，```head -n -M file; tail -n +(M+1) file; tail -f growing_file```</b>
* <b>pushd和popd：三个文件以上辅助定位，```pushd /var/www; dirs; pushd +3; popd +num```，将/var/www推到栈中，dirs显示栈中所有的目录，pushd +n切换到栈中的第n个目录，popd +n将栈中第n个目录删除</b>
* wc：统计文件长度、字符数、单词树，```wc -lwc file```
* tree：将文件目录结构以树状结构显示，```tree path -P PATTERN; tree path -I PATTERN; tree -h; tree PATH -H http://localhost -o out.html```
* ```crypt, gpg, base64, md5sum, sha1sum, openssl```：加密工具集
    * ```crypt < input_file >output_file```
    * ```gpg -c filename```
    * ```base64 filename > outputfile```
    * ```md5sum file```
    * ```sha1sum file```
* ```sort, uniq```
    * ```sort -nrk 1 data.txt```
    * ```sort unsorted.txt | uniq -c```

****

# 打包、解包、压缩、解压缩文件
* <b>tar：打包文件，默认打包格式为tar，参数选项```A, a, C, c, d, r, u, X, x, f, v, t, j, z, lzma```</b>
    * -v或-vv：打印详细信息
    * -c：创建新的打包文件```tar -cf archive.tar hello.txt```
    * -f：指定文件名
    * -r：像存在的打包文件中添加文件```tar -rvf original.tar new_file```
    * -t：列出打包文件中的所有文件
    * -x：解包打包文件```tar -xf archive.tar -C /path/to/extraction_directory```
    * -u：将文件发生变化的部分加入到压缩文件中，时候压缩增量日志文件压缩```tar -uf archive.tar filea```
    * -d：比较打包文件中的文件与文件系统中的文件差别
    * -j：指定bunzip2格式
    * -z：指定gzip格式
    * --lzma：指定lzma格式
    * -a或--auto-compress：根据文件名自动选择压缩格式```tar acvf archive.tar.gz filea fileb filec```
    * --delete：删除打包文件中的文件```tar --delete --file archive.tar [FILE LIST]```
    * --exclude：打包文件时排除相关文件```tar -cf arch.tar * --exclude "*.txt"```
    * --exclude-vcs：排除版本控制文件```tar --exclude-vcs -czvvf source_code.tar.gz eye_of_gnome_svn```
    * --totals：打印打包文件大小
    * -X：在使用exclude时候可以将排除文件放入列表中```tar -cf arch.tar * -X list```
    * -C：指定打包或者解包文件路径
    * -A：合并多个打包文件```tar -Af file1.tar file2.tar```
    * ```tar cvf - files/ | ssh user@example.com "tar xv -C Documents/"```：将压缩文件输出到stdout中，-代表stdout，作为管道流的输入
* cpio：-o指定了输出，-v用来打印打包文件列表，-i用于指定输入，-t表示列出打包文件中的内容，-d用来表示提取
    * ```cpio -it < archive.cpio```
    * ```cpio -id < archive.cpio```
* <b>zip, unzip：打包、解包文件，默认打包格式为zip</b>
    * ```zip -r archive.zip folder1 folder2```：递归打包文件
    * ```zip file.zip -u newfile```：更新打包文件的内容
    * ```zip -d arc.zip file.txt```：删除打包文件中的文件
    * ```unzip -l archive.zip```：列出打包文件中的文件
* <b>gzip, gunzip：压缩、解压文件，只能压缩单个文件，无法处理目录，多个文件需要配合tar使用</b>
    * ```gzip filename```，```gunzip filename.gz```
    * -c：将输出流重定向到stdout
    * --fast：指定最低压缩比例，速度最快
    * --best：指定最高压缩比例，速度最慢
    * -n 指定压缩比例，```gzip -5 test.img```
* zcat：指定读取gzip压缩过的文件内容```zcat test.gz```
* <b>bzip2, bunzip2：另一种压缩算法，用法与gzip, gunzip一致，但是压缩比例更高</b>
    * ```bzip2 filename```，```bunzip2 filename.bz```
* lzma, unlzma：另一种压缩算法，压缩比例最高
    * ```lzma filename```，```unlzma filename.lzma```
* pbzip2：可以指定多个cpu完成压缩，耗时可以考虑这个
    * ```pbzip2 -dc myfile.tar.bz2 | tar x```，解压的同时解包
    * ```pbzip2 myfile.tar```
* rsync：备份、恢复文件
    * ```rsync -av source_path destination_path```
    * ```rsync -av source_dir username@host:PATH```
    * ```rsync -av username@host:PATH destination```

****

# 管道流操作，也可以操作文件
* <b>xargs：命令把从stdin接收到的数据重新格式化，再将其作为参数提供给其他命令</b>
    * ```cat example.txt | xargs -n 3```
    * ```echo "splitXsplitXsplitXsplit" | xargs -d X -n 2```
    * ```cat args.txt | xargs -n 1 ./cecho.sh```
    * ```cat args.txt | xargs -I {} ./cecho.sh -p {} -l```：{}会替代为文件中的内容
    * ```find . -type f -name "*.txt" -print | xargs rm -f```
* ```tr [options] set1 set2```：将输入源中set1中的内容全部替换为set2
* <b>grep：通过正则表达式，搜索过滤输入信息，并将结果输出</b>
    * ```grep pattern filename; grep "pattern" filename```：通过pattern匹配输入内容
    * ```grep -E "[a-z]+" filename; egrep "[a-z]+" filename```：通过正则表达式匹配输入内容
    * ```grep word filename --color=auto```：着重标记匹配的内容
    * ```egrep -o "[a-z]+\."```：只输出匹配的内容
    * ```grep -v match_pattern file```：只输出不匹配的内容
    * ```grep -c "text" filename```：输出文件中匹配的字符串行数
    * ```grep linux -n```：输出匹配内容出现在文件中的行号
    * ```grep -l linux sample1.txt sample2.txt```：输出匹配内容出现在那些文件中
    * ```grep "text" . -R -n```：在多级目录中递归搜索
    * ```grep -e "pattern1" -e "pattern"```：指定多个匹配规则
    * ```grep "main()" . -r --include *.{c,cpp} ```：指定文件搜索
    * ```grep "main()" . -r --exclude "README"```：排除文件搜索
    * ```grep "test" file* -lZ | xargs -0 rm```：通过-Z指定匹配结果以\0结尾，-0表示xargs以\0作为分隔符
* <b>```find base_path options search_key command```：在base_path结合option搜索search_key的文件，并执行command</b>
    * ```-name, -iname```：按文件名搜索，search_key可以是正则表达式
        * ```find /home/slynux -name "*.txt" -print```
        * ```find . -iname "example*" -print```
        * ```find . \( -name "*.txt" -o -name "*.pdf" \) -print```：逻辑并操作，结合两个条件
    * ```-path```：按路径名搜索，search_key可以是正则表达式
        * ```find /home/users -path "*/slynux/*" -print```：根据路径匹配
    * ```-regex, -iregex```：按正则表达式搜索
        * ```find . -regex ".*\(\.py\|\.sh\)$"$```
        * ```find . -iregex ".*\(\.py\|\.sh\)$"$```
    * ```!```：对匹配条件取反，用排除法匹配，```find . ! -name "*.txt" -print```
    * ```-maxdepth, -mindepth```：指定目录深度搜索，如果指定这个要放在第三个参数，先确定深度在过滤其他
        * ```find . -maxdepth 1 -name "f*" -print ```
        * ```find . -mindepth 2 -name "f*" -print```
    * ```-type```：按文件类型搜索，文件类型有```f, l, d, c, b, s, p```
        * ```find . -type d -print```
    * ```-atime, -mtime, -ctime```：按文件操作时间搜索，时间单位是天，分别对应访问时间，修改时间，变化时间
        * ```find . -type f -atime -7 -print```：-7表示最近7天，+7表示超过7天，7表示正好是第7天
    * ```-amin, -mmin, -cmin```：按文件操作时间搜索，时间单位是天，分别对应访问时间，修改时间，变化时间
        * ```find . -type f -amin +7 -print```：-7表示最近7分钟，+7表示超过7分钟，7表示正好是第7分钟
    * ```-size```：按文件大小搜索，单位有```b, c, w, k, M, G```
        * ```find . -type f -size +2k```：+2表示大于2k，-2表示小于2k，2表示正好等于2k
    * ```-perm```：按文件权限搜索，```find . -type f -perm 644 -print```
    * ```-user```：按文件拥有者查找，```find . -type f -user root```
    * ```-exec```：对匹配的文件执行对应的命令
        * ```find . -type f -user root -exec chown slynux {} \; ```：{}代表匹配的文件，也就是会对所有的文件执行该命令
    * ```-prune```：排查指定文件，过滤部分匹配文件，```find devel/source_path \( -name ".git" -prune \) -o \( -type f -print \)```
* <b>sed指令（将输入源加载到一个模式空间进行操作）：流编辑器，操作、过滤、和转换文本的强大工具，可以将文件和管道作为输入源</b>
    * 默认该命令不会修改原始文件，-i选项表示直接修改原始文件，-ibak选项表示修改内容输出到bak文件中
    * ```sed [options] '{[address-range|pattern-range] sed-commands/[substitute-flags]}' {input-file}```：sed首先从input-file中读取第一行，然后执行所有的sed-commands；再读取第二行，执行所有sed-commands，重复这个过程，直到input-file结束；通过制定options还可以给sed传递一些可选的选项
        * options选项集介绍：
            * ```-n, -f, -e, -i, -ibak, -c, -l```
        * ranges标志介绍：
            * address-range：指文件的行数，例如[n,m]表示第n行到m行
            * pattern-range：指字符串模式，例如[/Raj/,/Jane/]表示匹配字符串Raj到字符串Jane
        * sed-commands命令集介绍：
            * 基础命令：```p, d, w, s```，分别是将模式空间的内容打印、删除、写入文件（注意w命令后面要接输出文件名），可以配合其他命令一起使用（将其他命令处理完的结果交给这三个命令处理）
            * ```#!/bin/sed -f```在文件开头标明该行，将sed当脚本文件执行
            * 修改命令：```a, i, c```
            * 格式化打印命令：```l, =```
            * 转换命令：```y```，转换命令是以字符为单位，替换命令是以单词为单位
            * 高级命令：```x, h, H, g, G, N, P, D, b, t```
        * substitute-flags介绍：
            * ```g, num, p, i, w, e, &```，p表示全局匹配，num表示只替换第几次出现的内容，&标志要结合正则表达式使用，e表示将模式空间中匹配的内容当shell命令执行
        * 具体示例：
            * ```sed –n –f test-script.sed /etc/passwd```：将sed-commands存放在test-script.sed中批量执行
            * ```sed –n –e '/^root/ p' –e '/^nobody/ p' /etc/passwd```：通过-e执行多个sed-commands
            * ```sed –n '{/^root/ p /^nobody/ p}' /etc/passwd```：通过{}分组命令执行
            * ```sed -n 'n,m p' employee.txt```：打印第n行到第m行的内容
            * ```sed -n '/Raj/,/Jane/ p' employee.txt```：打印匹配Raj的行起到匹配Jane的行结束
            * ```sed 's/original-string/replacement-string' input file```：替换操作，其中```/```分隔符可以替换为任意的```|, ^, @, !```
            * ```sed 'y/abcde/ABCDE/' employee.txt```：将abcde全部替换为ABCDE
            * ```sed 's/^[0-9][0-9][0-9]/[&]/g' employee.txt```：&表示匹配内容，该命令表示给匹配内容加上[]
* <b>awk指令处理有一定格式的文本数据文件：</b>
    * ```awk –F '/pattern/ {action}' input-file```或者```awk –F '{action}' input-file```或者```awk 'BEGIN { FS=":";print "----header----" } /mail/ {print $1$} END {print "----footer----"}' /etc/passwd```
        * F表示分隔符，默认为空格；```/pattern/```和```{action}```需要用单引号引起来；```{action}```必须用花括号括起来
        * BEGIN、END区域只执行一次
        * ```/pattern/```：模式匹配，过滤不需要处理的行数据
        * ```{action}```：命令集，需要对行数据执行的命令
        * 内置常量和变量：
            * ```FS, OFS, RS, ORS, NR, FILENAME, FNR```：指定文件分隔符，输出文件分隔符等
            * ```asort, asorti, print, printf, int, log, sqrt, exp, sin, cos, atan2, rand, srand, getline```：内置函数
            * ```index, length, split, substr```：字符串函数
            * ```sub, gsub, match, tolower, toupper, systime, strftime```：GAWK/NAWK才能使用的字符串函数
            * ```ARGC, ARGV, ARGIND, OFMT, ENVIRON, IGNORECASE, ERRNO```：GAWK/NAWK才能使用的内置变量
            * 普通变量，操作符语法，条件分支，循环，函数，位操作与C语言一致，可以像C一样编程
    *  pgawk，awk指令分析器，```pgawk -f profier.awk items.txt;  pgawk --profile=myprofiler.out -f profier.awk items.txt```
        * 示列：``` awk -F: '/mail/ {print $1$}' /etc/passwd```

****

# 网络操作
* <b>wget：用于文件下载的命令行工具</b>
    * ```wget URL1 URL2 URL3```
    * ```wget ftp://example_domain.com/somefile.img -O dloaded_file.img -o log```：-O指定输出文件名，-o指定一个日志文件
    * ```wget -t 5 URL```：-t指定重试次数
    * ```wget --limit-rate 20k http://example.com/file.iso```：--limit-rate对wget进行限速
    * ```wget -Q 100m http://example.com/file1 http://example.com/file2```：--quota或-Q指定最大下载配额
    * ```wget -c URL```：-c从断点开始继续下载
    * ```wget --mirror --convert-links exampledomain.com```：--mirror以递归的方式遍历网页上所有的URL链接，复制整个网站
    * ```wget -r -N -l -k DEPTH URL```：-l指定页面层级，-r递归选项，-N表示使用文件的时间戳，-k或–convert-links指示wget将页面的链接地址转换为本地地址
    * ```wget --user username --password pass URL```：下载需要HTTP或FTP认证的URL，--ask-password由网页提示并手动输入密码
* lynx：获取纯文本形式的网页
    *  ```lynx URL -dump > webpage_as_text.txt```
* <b>curl：发生网络请求</b>
    * ```curl URL --silent -o new_filename```
    * ```curl -C - URL```：自动断点续传
    * ```curl --referer Referer_URL target_URL```：指定参照页去请求目标页
    * ```curl http://example.com --cookie "user=slynux;pass=hack"```：指定cookie发生请求
    * ```curl URL --cookie-jar cookie_file```：将请求回来的cookie打包成文件
    * ```curl URL --user-agent "Mozilla/5.0"```：指定浏览器代理
    * ```curl -H "Host: www.slynux.org" -H "Accept-language: en" URL```：指定请求头部
    * ```curl URL --limit-rate 20k```：--limit-rate对curl进行限速
    * ```curl URL --max-filesize bytes```：指定最大下载量
    * ```curl -u user:pass http://test_auth.com```：-u完成HTTP或FTP认证
    * ```curl -I http://slynux.org```：只打印头部信息
    * ```curl URL -d "postvar=postdata2&postvar2=postdata2"```：以post请求发送数据访问网站

****

# 网络配置
* <b>ifconfig：配置及显示网络接口、子网掩码等详细信息</b>
    * ```ifconfig wlan0 192.168.0.80 netmask 255.255.252.0```：手动设置网络接口的IP地址和子网掩码
    * ```dhclient eth0```：动态配置网络
    * ```ifconfig eth0 hw ether 00:1c:bf:87:25:d5```：手动修改硬件MAC地址，重启后失效
* iwconfig, iwlist：无线网络管理
* host, nslookup：DNS查找命令
    * ```host google.com```
    * ```nslookup google.com```
* route：路由表管理
* traceroute：检查包转发的跳数，检查网络的利器
* ping：网络连通性检查，-c指定ping的次数
* fping：同时ping一组ip地址，-a指定打印出所有活动主机的IP地址，-u指定打印出所有无法到达的主机，-g指定从"IP地址/子网掩码"记法或者"IP地址范围"记法中生成一组IP地址```fping -a 192.160.1/24 -g```
* <b>ssh：远程登录主机</b>
    * ```ssh username@remote_host```
    * -p：指定连接端口，```ssh user@locahost -p 422```
    * ```ssh user@host "command1 ; command2 ; command3"```：在远端机器执行命令，```ssh mec@192.168.0.1 "echo user: $(whoami);echo OS: $(uname)"```
    * ```ssh -L 8000:www.kernel.org:80 user@localhost```：将本机8000端口的流量转发到www.kernel.org:80端口中
    * ```ssh -L 8000:www.kernel.org:80 user@REMOTE_MACHINE```：将远端8000端口的流量转发到www.kernel.org:80端口中
    * ```ssh -R 8000:localhost:80 user@REMOTE_MACHINE```：将远程主机端口8000上的流量转发到本地主机的端口80上
* <b>lftp, sftp, scp：通过网络传输文件</b>
    * lftp和sftp是交互式命令
    * ```scp SOURCE DESTINATION```：```scp user@remotehost:/home/path/filename filename```，可以通过-oPort指定端口，-r递归传递文件夹
* ssh-keygen：生成ssh公钥和密钥```ssh-keygen -t rsa```
* <b>sshfs：将远端服务器的目录挂在到本地```sshfs -o allow_other user@remotehost:/home/path /mnt/mountpoint```</b>
* lsof：列出本机已经打开的端口以及运行在端口上的服务，```lsof -i | grep ":[0-9]\+->" -o | grep "[0-9]\+" -o | sort | uniq```
* netstat：查看开放端口与服务```netstat -tnp```
* nc：创建套接字```nc -l 1234; nc HOST 1234```
* iptables：过滤阻塞流量，允许正常流量通过
    * ```iptables -A OUTPUT -d 8.8.8.8 -j DROP```：将发送到特定ip的流量都丢弃
    * ```iptables -A OUTPUT -p tcp -dport 21 -j DROP```：将发送到特定端口的流量都丢弃
    * ```iptables --flush```：对新添加的规则生效
    * -A：表示向链中添加一条规则
    * OUTPUT：表示流量出口链
    * -d：指定了所要匹配的分组目的地址
    * -j DROP：表明行动，丢弃包
    * -p：指定该规则是适用于TCP
    * -dport：指定了对应的端口

****

# 系统管理
* /etc/passwd：查看用户基本信息
* /etc/group：查看用户组信息
* /proc：该文件夹下包含系统很多详细信息
* /proc/PID/environ：查看进程使用的环境变量
* /proc/cpuinfo：查看cpu信息
* /proc/meminfo：查看内存信息
* /proc/partitions：查看分区信息
* ```echo -e "\e[1;31m This is red text \e[0m" ```：将文字按红色打印
* fork：函数在父进程中返回子进程的ID，在子进程中返回0
* tput和stty：终端处理命令
* date：系统日期时间命令
* <b>time：获取命令执行时间</b>
    * ```/usr/bin/time -a -o output.txt COMMAND```：将统计结果输出到文件中
    * ```/usr/bin/time -f "FORMAT STRING" COMMAND```：格式化时间输出
        * ```/usr/bin/time -f "Time: %U" -a -o timing.log uname```
        * %C 进行计时的命令名称以及命令行参数
        * %D 进程非共享数据区域的大小，以KB为单位
        * %E 进程使用的real时间（挂钟时间），显示格式为[小时:]分钟:秒
        * %x 命令的退出状态
        * %k 进程接收到的信号数量
        * %W 进程被交换出主存的次数
        * %Z 系统的页面大小。这是一个系统常量，但在不同的系统中，这个常量值也不同
        * %P 进程所获得的CPU时间百分比。这个值等于user+system时间除以总运行时间。结果以百分比形式显示
        * %K 进程的平均总（data+stack+text）内存使用量，以KB为单位
        * %w 进程主动进行上下文切换的次数，例如等待I/O操作完成
        * %c 进程被迫进行上下文切换的次数（由于时间片到期）
* <b>ps：收集进程信息</b>
    * -f：显示多列，```ps -f```
    * -e：显示所有进程，```ps –ef```
    * -ax：显示所有进程，```ps –axf```
    * -o：显示需要的列，```ps -eo comm,pcpu```
        * pcpu CPU占用率
        * pid 进程ID 
        * ppid 父进程ID 
        * pmem 内存使用率
        * comm 可执行文件名
        * cmd 简单命令
        * user 启动进程的用户
        * nice 优先级
        * time 累计的CPU时间
        * etime 进程启动后流逝的时间
        * tty 所关联的TTY设备
        * euid 有效用户ID 
        * stat 进程状态
    * --sort：按不同的列排序，+升序，-降序，```ps -eo comm,pcpu --sort -pcpu```
    * -t：指定进程所属的TTY，```ps -t TTY1, TTY2```
    * -L：输出进程拥有的线程信息，```ps -eLf --sort -nlwp```
    * ```ps -C COMMAND_NAME -o pid=```：获得特定命令的进程ID列表，可用pgrep指令替代
* <b>top：显示占用CPU最多的进程外，还会显示进程相关的一些其他参数</b>
* pgrep：获得特定命令的进程ID列表
    * ```pgrep bash -d ":"```：指定输出分隔符
    * ```pgrep -u root,slynux COMMAND```：指定进程的用户（拥有者）列表
* <b>kill, killall：结束进程，向进程发送信号</b>
    * ```kill PROCESS_ID_LIST```：想进程发送TERM信号结束进程
    * ```kill -s SIGNAL PID```：想进程发送信号，通过```kill -l```查看支持的信号
        * SIGHUP 1——对控制进程或终端的终结进行挂起检测（hangup detection）。
        * SIGINT 2——当按下Ctrl + C时发送该信号。
        * SIGKILL 9——用于强行杀死进程。
        * SIGTERM 15——默认用于终止进程。
        * SIGTSTP 20——当按下Ctrl + Z时发送该信号
        * ```kill -s SIGKILL PROCESS_ID```，```kill -9 PROCESS_ID```
* trap：在shell脚本中接收kill发送的信号，并通过函数处理该信号
    * ```trap 'signal_handler_function_name' SIGNAL LIST```：signal_handler_function_name就是处理函数名
* <b>du：disk usage的缩写，查看文件占用的磁盘量</b>
    * ```du FILENAME1 FILENAME2```
    * ```du -a DIRECTORY```：-a递归地输出指定目录或多个目录中所有文件的统计结果
    * ```du -h FILENAME ```：更友好的格式进行打印
    * ```du -c FILENAME1 FILENAME2```：在最后显示总的磁盘占有量
    * ```du -s FILES(s)```：只显示总使用量
    * ```du -b FILE(s)```：使用特定单位显示，单位选项包括```-b, -k, -m, -B```
    * ```du --exclude "*.txt" FILES(s)```：排除文件不计入统计
    * ```du --exclude-from EXCLUDE.txt DIRECTORY```：排除文件列表不计入统计
    * ```du --max-depth 2 DIRECTORY```：指定统计深度
* <b>df：disk free的缩写，查看磁盘可用空间信息，用法与du一样</b>
* who, w, users：获取当前用户登录信息
* uptime：查看当前系统开机运行多久了
* last：获取上一次启动以及用户登录会话的信息
* <b>watch：监控某个命令持续执行的结果</b>
    * ```watch 'COMMANDS'```
    * ```watch -n 5 'ls -l'```：每隔5秒输出一次执行命令结果
* inotifywait：查看文件操作历史记录
* logrotate：日志管理工具
* logger：通过syslogd记录日志
    * /var/log/boot.log 系统启动信息
    * /var/log/httpd Apache Web服务器日志
    * /var/log/messages 发布内核启动信息
    * /var/log/auth.log 用户认证日志
    * /var/log/dmesg 系统启动信息
    * /var/log/mail.log 邮件服务器日志
    * /var/log/Xorg.0.log X服务器日志
* iotop：监视磁盘I/O
* fsck：检查文件系统错误以及磁盘分区等，是交互型命令
* which：查找指令的位置，``` which ls```
* whereis：功能与which一样，更强大，可以打印出命令手册的位置，```whereis ls```
* file：查看文件类型，```file /bin/ls```
* whatis：解释命令，```whatis ls```
* hostname, uname：查询系统相关信息
* cron job：定时执行任务，```02 * * * * /home/slynux/test.sh```
* useradd, deluser, chsh, usermod, chage, passwd, addgroup, delgroup：用户账号处理
* convert：将一种图片转化为另一种图片格式
    * ```convert INPUT_FILE OUTPUT_FILE```
    * ```convert image.png -resize WIDTHxHEIGHT image.png```
    * ```convert image.png -resize WIDTHx image.png```
    * -source：用于指定图像源目录
    * -percent：用于指定缩放比例
    * -scale：用于指定缩放宽度与高度
    * -ext：用于指定目标文件格式
    * -dest：为缩放或转换格式后的文件指定目标目录
* <b>vmstat和iostat</b>：
<table>
    <tr>
        <td><img src='../images/mysql/vmstat.png' width='450px'></td>
        <td><img src='../images/mysql/vmstat参数1.png' width='450px'></td>
    </tr>
     <tr>
        <td><img src='../images/mysql/vmstat参数2.png' width='450px'></td>
        <td><img src='../images/mysql/iostat.png' width='450px'></td>
    </tr>
    <tr>
        <td><img src='../images/mysql/iostat参数1.png' width='450px'></td>
        <td><img src='../images/mysql/iostat参数2.png' width='450px'></td>
    </tr>
    <tr>
        <td><img src='../images/mysql/iostat参数3.png' width='450px'></td>
    </tr>
</table>

****

# shell编程
* ```read -n number_of_chars variable_name```：读取输入源到变量，```read -n 2 var; read -s var; read -p "Enter input:" var; read -t timeout var; read -d delim_char var```
* 变量相关
    * ```${VAR%.*}, ${VAR%%.*}, ${VAR#.*}, ${VAR##.*}```：拆分文件名和后缀，%表示将VAR从右边开始匹配```.*```表达式的内容删除，#则是从左边开始
    * ```$Var, ${Var}, length=${#var}$```：都是使用变量的语法，无区别；最后为获取变量长度
    * ```HOME、PWD、USER、UID、SHELL```：系统环境变量
    * ```export PATH="$PATH:/home/user/bin$"```：导入环境变量
    * ```echo $SHELL, echo $0```：查看当前使用的shell
    * ```array_var=(1 2 3 4 5 6); ${array_var[$index]}; ${array_var[*]}; ${array_var[@]}; ${#array_var[*]}$ ```：数组
    * ```declare -A fruits_value; fruits_value=([apple]='100dollars' [orange]='150 dollars'); ${fruits_value[apple]}; ${!fruits_value[*]}; ${!fruits_value[@]}$```：关联数组map
* 函数、循环、条件分支
    * ```function fname() { statements; }```或```fname() { statements; } ```：定义函数；```fname arg1 arg2 ;```调用函数；```$n, "$@", "$*"$```：分别是第n个参数，以列表的形式访问所有参数，将参数合并成一个的字符串；```export -f fname```将函数导出，以便子进程使用；```$?$```获取命令函数执行返回值
    * ```for var in list; do commands; done```或```while [condition] do commands; done```或```until [ condition ]; do commands done ```：循环语句
    * ```if condition; then commands; fi```或```if condition; then commands; else if condition; then commands; else commands; fi ```：条件分支语句
        * ```[ $var1 -ne 0 -a $var2 -gt 2 ]```：逻辑与
        * ``` [[ -n $str1 ]] && [[ -z $str2 ]]```：逻辑与
        * ```[ $var1 -ne 0 -o $var2 -gt 2 ]```：逻辑或
        * ``` [[ -n $str1 ]] || [[ -z $str2 ]]```：逻辑或
        * ```[ -f $file_var$ ]```：如果给定的变量包含正常的文件路径或文件名，则返回真。
        * ```[ -x $var$ ]```：如果给定的变量包含的文件可执行，则返回真。
        * ```[ -d $var$ ]```：如果给定的变量包含的是目录，则返回真。
        * ```[ -e $var$ ]```：如果给定的变量包含的文件存在，则返回真。
        * ```[ -c $var$ ]```：如果给定的变量包含的是一个字符设备文件的路径，则返回真。
        * ```[ -b $var$ ]```：如果给定的变量包含的是一个块设备文件的路径，则返回真。
        * ```[ -w $var$ ]```：如果给定的变量包含的文件可写，则返回真。
        * ```[ -r $var$ ]```：如果给定的变量包含的文件可读，则返回真。
        * ```[ -L $var$ ]```：如果给定的变量包含的是一个符号链接，则返回真。
        * ```[[ $str1 = $str2 ]]```：当str1等于str2时，返回真。也就是说，str1和str2包含的文本是一模一样的。
        * ```[[ $str1 == $str2 ]]```：这是检查字符串是否相等的另一种写法
        * ```[[ $str1 > $str2 ]]```：如果str1的字母序比str2大，则返回真。
        * ```[[ $str1 < $str2 ]]```：如果str1的字母序比str2小，则返回真。
        * ```[[ -z $str1$ ]]```：如果str1包含的是空字符串，则返回真。
        * ```[[ -n $str1$ ]]```：如果str1包含的是非空字符串，则返回真。
* ```alias new_command='command sequence'; alias install='sudo apt-get install'```：设定命令别名，将别名加入到.bashrc或.zshrc才会永久生效
* ```bash -x script.sh```：调试shell脚本
* ```IFS```：改变字符串中的分隔符，在循环遍历中使用
* \> 和 >>：重定向输出源，>是先清空文件，然后再写入内容，>>将内容追加到现有文件的尾部
* < 和 <<：重定向输入源
* 正则表达式（[可视化正则表达式](http://www.regexper.com), [调试正则表达式](https://regexr.com/)）：<img src='../images/devops/正则表达式.png' width='450px'>